# Siamese Network

#### dependecies

In [1]:
%matplotlib inline
import random
from skimage import io
import matplotlib.pyplot as plt
from ipywidgets import interact
from keras.models import Model, Sequential 
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
import numpy as np
from keras.layers import Input, Flatten, Dense, Dropout, Conv2D, MaxPooling2D, Merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.layers.core import Lambda
import keras
import os
from sklearn.metrics import accuracy_score
import seaborn as sns
import pandas as pd
import sys
sys.path.append("../")
from networks.networks import *

CSV_DIR = '../data/csv/'
MODEL_DIR = '../models/'
LOGS_DIR = '../logs'

Using TensorFlow backend.


### Load Data

In [2]:
train_df = pd.read_csv(os.path.join(CSV_DIR, 'train_pairs.csv'))
test_df = pd.read_csv(os.path.join(CSV_DIR, 'test_pairs.csv'))

In [3]:
# train_df.head(20)

In [4]:
X1_paths = list(train_df.X1[0:20])
X2_paths = list(train_df.X2[0:20])

In [5]:
X1_test_paths = list(test_df.X1[0:10])
X2_test_paths = list(test_df.X2[0:10])

In [6]:
X1 = np.array(io.imread_collection(X1_paths))
X2 = np.array(io.imread_collection(X2_paths))

In [7]:
X1_test = np.array(io.imread_collection(X1_test_paths))
X2_test = np.array(io.imread_collection(X2_test_paths))

In [8]:
y = np.array(train_df.y)
y_test = np.array(test_df.y)

In [9]:
X1.shape, X2.shape, y.shape

((20, 230, 105, 3), (20, 230, 105, 3), (28733,))

In [10]:
X1_test.shape, X2_test.shape, y_test.shape

((10, 230, 105, 3), (10, 230, 105, 3), (14554,))

#### Loss and distance related Functions

In [11]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.mean(K.square(x - y), axis=1, keepdims=True), K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    threshold = 0.2
    return 1 -  K.mean(K.equal(y_true, K.cast(y_pred < threshold, y_true.dtype)))


def accuracy5(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    threshold = 0.5
    return 1 -  K.mean(K.equal(y_true, K.cast(y_pred < threshold, y_true.dtype)))

def accuracy1(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    threshold = 1.0
    return 1 -  K.mean(K.equal(y_true, K.cast(y_pred < threshold, y_true.dtype)))


def contrastive_loss(y_diff, y_dist):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1.0
    return K.mean((1 - y_diff) * K.square(y_dist) +
                  (y_diff) * K.square(K.maximum(margin - y_dist, 0)))


### Build Network Architecture

In [12]:
network = base_network('alex_net')

In [13]:
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 220, 95, 32)       11648     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 109, 47, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 101, 39, 64)       165952    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 93, 31, 64)        331840    
_________________________________________________________________
dropout_1 (Dropout)          (None, 93, 31, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 93, 31, 64)        256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 16, 128)       401536    
__________

### Configure Siamese Network

In [32]:
input_a = Input(shape=[230, 105, 3])
input_b = Input(shape=[230, 105, 3])

In [33]:
processed_a = network(input_a)
processed_b = network(input_b)

#### Merge networks with euclidean distance

In [34]:
distance = Lambda(euclidean_distance,
                          output_shape=eucl_dist_output_shape)([processed_a, processed_b])

In [35]:
model = Model([input_a, input_b], distance)

In [36]:
# model.save(os.path.join(MODEL_DIR, "train_4.h5"))

#### Configure loss and other parameters

In [37]:

model.compile(loss=contrastive_loss, optimizer='SGD', metrics=[accuracy, accuracy1, accuracy5])

In [38]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 230, 105, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 230, 105, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 64)           35171392    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           sequential_1[1][0]               
          

#### Set-up TensorBoard

In [39]:
# tf_board = TensorBoard(os.path.join(LOGS_DIR, 'logs', 'exp_5'))

In [40]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, verbose=1, mode='auto')

### Train Siamese Network 

In [43]:
history = model.fit([X1, X2], y[0:20], 
                    batch_size=1, epochs=10, verbose=1, callbacks=[earlystop],
                    validation_data=([X1_test, X2_test], y_test[0:10]))

Train on 20 samples, validate on 10 samples
Epoch 1/10
20/20 [==============================] - 2s 119ms/step - loss: 0.2816 - accuracy: 0.5000 - accuracy1: 0.7000 - accuracy5: 0.6500 - val_loss: 6.0813 - val_accuracy: 0.4000 - val_accuracy1: 0.8000 - val_accuracy5: 0.7000
Epoch 2/10
 5/20 [======>.......................] - ETA: 0s - loss: 0.0982 - accuracy: 0.6000 - accuracy1: 0.8000 - accuracy5: 1.0000

/home/jchan/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:526: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,val_accuracy1,val_accuracy5,loss,accuracy,accuracy1,accuracy5
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


20/20 [==============================] - 1s 33ms/step - loss: 4.5992 - accuracy: 0.5000 - accuracy1: 0.5000 - accuracy5: 0.6000 - val_loss: 2.4668 - val_accuracy: 0.4000 - val_accuracy1: 0.4000 - val_accuracy5: 0.4000
Epoch 3/10
20/20 [==============================] - 1s 34ms/step - loss: 1.3127 - accuracy: 0.5500 - accuracy1: 0.7500 - accuracy5: 0.7000 - val_loss: 1.0932 - val_accuracy: 0.4000 - val_accuracy1: 0.8000 - val_accuracy5: 0.4000
Epoch 4/10
20/20 [==============================] - 1s 34ms/step - loss: 1.7318 - accuracy: 0.5000 - accuracy1: 0.5500 - accuracy5: 0.6500 - val_loss: 0.2850 - val_accuracy: 0.6000 - val_accuracy1: 0.7000 - val_accuracy5: 0.5000
Epoch 5/10
20/20 [==============================] - 1s 33ms/step - loss: 2.3387 - accuracy: 0.6000 - accuracy1: 0.4000 - accuracy5: 0.6500 - val_loss: 1.3664 - val_accuracy: 0.4000 - val_accuracy1: 0.8000 - val_accuracy5: 0.4000
Epoch 6/10
20/20 [==============================] - 1s 34ms/step - loss: 15.8299 - accuracy: 0.

In [ ]:
# model.save("../models/exp_5.h5")
# model.save_weights("../weights/exp_5-w.h5")

In [46]:
model.predict([X1_test, X2_test]), y_test[0:10]

(array([[  2.62811279e+00],
        [  3.16227757e-04],
        [  3.16227757e-04],
        [  3.16227757e-04],
        [  3.16227757e-04],
        [  3.16227757e-04],
        [  2.84238720e+00],
        [  9.13267016e-01],
        [  3.16227757e-04],
        [  1.19415932e+01]], dtype=float32),
 array([1, 0, 0, 1, 1, 0, 0, 0, 0, 1]))